Tutorial link : https://pyimagesearch.com/2020/08/10/opencv-sudoku-solver-and-ocr/

# image preprocessing

In [1]:
!pip install opencv-python


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install tensorflow==2.15


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import cv2
import tensorflow as tf

2024-07-30 16:19:17.780895: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 16:19:17.822324: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 16:19:18.073157: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 16:19:18.073316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 16:19:18.108196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [5]:
image_path = "test1.png"

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(image, (5, 5), 0)
# Apply adaptive thresholding
thresholded_img = cv2.adaptiveThreshold(
    blurred, 
    255, 
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV, 11, 2
)

cv2.imwrite( "xyz.png" ,thresholded_img)

True

# find largest square

# warp image

# divide image into 9*9 grids

In [6]:
thresholded_img.shape

(571, 570)

In [7]:
cells = []
image = thresholded_img

h, w = image.shape
cell_h, cell_w = h//9, w//9

for i in range(9):
    row = []
    for j in range(9):
        cell = image[i*cell_h:(i+1)*cell_h, j*cell_w:(j+1)*cell_w]          
        row.append(cell)
    cells.append(row)

cells

[[array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255,   0, ..., 255, 255, 255],
         ...,
         [255, 255,   0, ...,   0,   0,   0],
         [255, 255,   0, ...,   0,   0,   0],
         [255, 255,   0, ...,   0,   0,   0]], dtype=uint8),
  array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255,   0,   0, ..., 255, 255, 255],
         ...,
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8),
  array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [  0,   0,   0, ...,   0,   0,   0],
         ...,
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8),
  array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255,

# digit detection

In [35]:
model = tf.keras.models.load_model("mnist_digits_cnn.keras")

In [12]:
print(cells[0][0].shape)

(63, 63)


In [36]:
sudoku_grid = []

for row in cells:
    sudoku_row = []
    for cell in row:
        #preprocess cell
        cell = cv2.resize(cell, (28, 28))
        cell = cell.astype("float32") / 255.0
        cell = np.expand_dims(cell, axis = -1)
        cell = np.expand_dims(cell, axis = 0)
        # print(cell.shape)

        #count the white filled area percentage
        percentFilled = cv2.countNonZero(cell) / float(28 * 28)
        
        #predict using cnn
        prediction = model.predict(cell)
        digit = np.argmax(prediction)

        #if < 20% white, ignore
        if(percentFilled > 0.2):
            sudoku_row.append(digit)
        else:
            sudoku_row.append(" ")
    
    sudoku_grid.append(sudoku_row)


1/1 [==============================] - 0s 14ms/step


In [37]:
for row in sudoku_grid:
    print(row)

[' ', ' ', ' ', 2, 6, ' ', 7, ' ', 7]
[6, 8, ' ', ' ', 7, ' ', ' ', 9, ' ']
[2, 9, ' ', ' ', ' ', 4, 5, ' ', ' ']
[8, 2, ' ', 2, ' ', ' ', ' ', 4, ' ']
[' ', ' ', 4, 6, ' ', 2, 9, ' ', ' ']
[' ', 5, ' ', ' ', ' ', 3, ' ', 2, 8]
[' ', ' ', 9, 3, ' ', ' ', ' ', 7, 4]
[' ', 4, ' ', ' ', 5, ' ', ' ', 3, 6]
[7, ' ', 3, ' ', 4, 8, ' ', ' ', ' ']
